In [ ]:
'''
wrangle-hsis-data.py
Usage: reads and merges HSIS data for designated years
'''

### import

In [51]:
import random
import time

import pandas as pd
import numpy as np
import sqlite3

from matplotlib import pyplot as plt
from matplotlib.pyplot import rc

import shapefile as shp
import folium

### NNNN - don't execute - Check coordinates
We can confirm 
+ HSIS data has more records than CSET, probably because CSET focuses on rural areas, while HSIS has both RURAL & URBAN;
+ HSIS coordinates are the same with CSET, no records have X, Y difference larger than 1e-9;


Output: **wa17acc_latlng.csv**. *No need to do this conversion step again*

#### read and merge HSIS and CSET

In [3]:
acc_file = './hsis/wa17acc.xlsx'
acc = pd.read_excel(acc_file)

columns = ['FORM_REPT_NO', 'ACCYR', 'State_Plane_X', 'State_Plane_Y']
acc = acc[columns]
acc.columns = ['REPORT NUMBER', 'YEAR', 'X_full', 'Y_full']
acc.head()

,REPORT NUMBER,YEAR,X_full,Y_full
0,E627527,2017,1041008.91,428924.73
1,E628124,2017,1475667.71,691747.63
2,E630351,2017,2424216.82,579569.52
3,E633194,2017,2475616.25,525827.20
4,E632431,2017,1742710.63,165973.98


In [ ]:
cset = pd.read_csv('../tech/WA_Rural_St_RtesCrashes_Full.csv')

cset['YEAR'] = pd.to_datetime(cset['DATE']).dt.year
columns = ['REPORT NUMBER', 'YEAR', 'WA STATE PLANE SOUTH - X 2010 - FORWARD', 'WA STATE PLANE SOUTH - Y 2010 - FORWARD']
cset = cset[columns]
cset.columns = ['REPORT NUMBER', 'YEAR', 'X_cset', 'Y_cset']

cset = cset[cset['YEAR'] == 2017].reset_index(drop=True)
print('Num of records in CSET 2017: {}\n'.format(cset.shape[0]))
cset.head(5)

In [ ]:
res = acc.merge(cset, on=['REPORT NUMBER', 'YEAR'], how='inner')
res.head()

#### No records have larger coords difference of X than 1e-9

In [ ]:
res[res['X_full'] - res['X_cset'] > 1e-9]

#### No records have larger coords difference of Y than 1e-9

In [ ]:
res[res['Y_full'] - res['Y_cset'] > 1e-9]

#### 2017 crashes has NaN values???
yes, discard them!

In [ ]:
acc[acc.isna().any(axis=1)].head()

#### output X,Y of HSIS for conversion
dropped 26 entries with missing X,Y info

In [ ]:
acc = acc.dropna(subset=['X_full', 'Y_full']).reset_index()

In [ ]:
output = acc[['Y_full', 'X_full']]
output.columns = ['northing', 'easting']
output['zone'] = 4602
output['units'] = 'usft'
output['inDatum'] = 'NAD83(2011)'
output['outDatum'] = 'NAD83(2011)'
output['utmZone'] = 'auto'
output['eht'] = 'N/A'
output['ID'] = output.index + 1

In [ ]:
output = output[['ID', 'zone', 'northing', 'easting', 'units', 'inDatum', 'outDatum', 'utmZone', 'eht']]
output

In [ ]:
for i in range(output.shape[0] // 4000):
    output[(i*4000):((i+1)*4000)].to_csv('coords_2017_{}.csv'.format(i), index=None, sep=',')
output[((i+1)*4000):].to_csv('coords_2017_{}.csv'.format(i+1), index=None, sep=',')

#### read in noaa conversion result

In [ ]:
columns = ['ID', 'destLat', 'destLon']

acc_coords = pd.read_csv('coords (1).csv')
acc_coords = acc_coords[columns]
for file in ['coords (2).csv', 'coords (3).csv', 'coords (4).csv']:
    tmp = pd.read_csv(file)
    tmp = tmp[columns]
    acc_coords = acc_coords.append(tmp).reset_index(drop=True)

In [ ]:
acc_coords.columns = ['ID', 'lat', 'lon']

In [ ]:
acc_coords.shape

#### incorporate noaa conversion

In [ ]:
acc_file = './hsis/wa17acc.xlsx'
acc = pd.read_excel(acc_file)

acc = acc.dropna(subset=['State_Plane_X', 'State_Plane_Y'])
acc['ID'] = acc.index + 1

In [ ]:
acc = acc.merge(acc_coords, on='ID', how='inner')

In [ ]:
acc.to_csv('wa17acc_latlng.csv')

In [ ]:
acc.columns

### read

#### milepost

In [ ]:
def plotMilePost(mile_shapefile, mapSaveLoc):
    '''
    @param mile_shapefile: shape file of original data
    @param mapSaveLoc: saving destination of generated folium map
    '''
    
    # read shape file
    milepost = shp.Reader(mile_shapefile)
    
    # read lat, lon
    # be careful which is lat, which is lng
    x = []
    y = []

    for shape in milepost.shapeRecords():
        for i in shape.shape.points[:]:
            x.append(i[0])
            y.append(i[1])

    # create map background
    waMilePost = folium.Map([np.median(np.array(y)), np.median(np.array(x))],
               # tiles="cartodbpositron",
               tiles='',
               # width='80%',
               # height='80%',
               prefer_canvas=True,
               zoom_start=7)
    
    folium.TileLayer('cartodbpositron', name = 'bright').add_to(waMilePost)
    
    # add layer of milepost points
    milepostLayer = folium.FeatureGroup(name='mileposts', show=False)
    waMilePost.add_child(milepostLayer)

    # plot the mileposts on the map
    # honestly, we don't knwo verbal info of the points
    for i in range(len(x)):
        folium.CircleMarker([y[i], x[i]],
                    radius=2,
                    popup=folium.Popup("milepost {}".format(3), max_width=150),
                    # fill_color="#3db7e4",
                    # color='red',
                    weight = 0.1,
                    fill_color='yaleblue',
                    fill=True,
                    fill_opacity=0.4
             ).add_to(milepostLayer)
    
    # add layer control
    folium.LayerControl().add_to(waMilePost)
    
    # save and return
    waMilePost.save(mapSaveLoc)
    
    return waMilePost

In [ ]:
_ = plotMilePost('./milepost/SRMilepostMarkers.shp', 'waMilePost.html')

#### accident

In [2]:
acc_file = 'wa17acc_latlng.csv'
acc = pd.read_csv(acc_file)
acc = acc.dropna(subset=['lat', 'lon'])

In [3]:
columns = ['CASENO', 'FORM_REPT_NO', 'rd_inv', 'milepost', 'RTE_NBR',
           'lat', 'lon', 
           'MONTH', 'DAYMTH', 'WEEKDAY', 
           'RDSURF', 'LIGHT', 'weather', 'rur_urb',
           'REPORT', 'SEVERITY']

In [4]:
acc = acc[columns]

In [5]:
acc.head()

,CASENO,FORM_REPT_NO,rd_inv,milepost,RTE_NBR,lat,lon,MONTH,DAYMTH,WEEKDAY,RDSURF,LIGHT,weather,rur_urb,REPORT,SEVERITY
0,2017000001,E627527,005,64.13,5,46.484755,-122.879959,1,1,7,2.0,6.0,01,R,1,1
1,2017000002,E628124,090,70.58,90,47.228310,-121.163097,1,2,1,4.0,2.0,08,R,2,7
2,2017000003,E630351,195,35.95,195,46.879918,-117.364820,1,9,1,4.0,1.0,02,R,1,1
3,2017000004,E633194,270,3.13,270,46.726902,-117.168536,1,9,1,3.0,1.0,04,U,1,1
4,2017000005,E632431,014,139.99,14,45.787757,-120.099029,1,14,6,3.0,6.0,02,R,2,5


In [6]:
acc.shape

(55494, 16)

#### curve

In [7]:
curv_file = './hsis-csv/wa17curv.csv'
curv = pd.read_csv(curv_file)

In [8]:
columns = ['curv_inv', 'begmp', 'endmp', 'rte_nbr', 'DIR_CURV', 'deg_curv']
curv = curv[columns]

In [9]:
curv.head()

,curv_inv,begmp,endmp,rte_nbr,DIR_CURV,deg_curv
0,002,0.16,0.19,2,L,43.74
1,002,0.20,0.29,2,R,17.74
2,002,0.32,0.79,2,L,0.93
3,002,1.84,1.92,2,R,0.60
4,002,2.37,2.72,2,R,3.82


#### grade

In [10]:
grad_file = './hsis-csv/wa17grad.csv'
grad = pd.read_csv(grad_file)

In [11]:
columns = ['grad_inv', 'begmp', 'endmp', 'rte_nbr', 'dir_grad', 'pct_grad']

In [12]:
grad = grad[columns]

In [13]:
grad.head()

,grad_inv,begmp,endmp,rte_nbr,dir_grad,pct_grad
0,002,0.00,0.16,2,-,2.19
1,002,0.16,0.18,2,-,3.87
2,002,0.18,0.25,2,-,0.19
3,002,0.25,0.30,2,+,0.44
4,002,0.30,0.37,2,+,2.76


#### occupant

In [14]:
occ_file = './hsis-csv/wa17occ.csv'
occ = pd.read_csv(occ_file)

In [15]:
occ.head()

,CASENO,SEATPOS,VEHNO
0,2017000004,7,1
1,2017000005,3,2
2,2017000011,3,2
3,2017000017,5,1
4,2017000017,6,1


#### road

In [16]:
road_file = './hsis-csv/wa17road.csv'
road = pd.read_csv(road_file)

In [17]:
columns = ['ROAD_INV', 'BEGMP', 'ENDMP', 'RTE_NBR', 'CITY', 'COUNTY',
           'AADT', 'mvmt', 
           'RURURB', 
           'LSHL_TYP', 'MED_TYPE', 'RSHL_TYP', 'SURF_TYP', 'SPD_LIMT',
           'EW_IND', 'LSHLDWID', 'MEDWID', 'RSHLDWID', 'lanewid', 'rdwy_wid', 'NO_LANES']
road = road[columns]

In [18]:
road.head(8)

,ROAD_INV,BEGMP,ENDMP,RTE_NBR,CITY,COUNTY,AADT,mvmt,RURURB,LSHL_TYP,...,RSHL_TYP,SURF_TYP,SPD_LIMT,EW_IND,LSHLDWID,MEDWID,RSHLDWID,lanewid,rdwy_wid,NO_LANES
0,002,0.00,0.04,2,420.0,31.0,6266.0,0.09,U,NaN,...,NaN,NaN,NaN,W,0,0,0,40,40,1
1,002,0.04,0.11,2,420.0,31.0,6266.0,0.16,U,NaN,...,NaN,NaN,55.0,W,0,0,0,20,40,2
2,002,0.11,0.13,2,420.0,31.0,9626.0,0.07,U,NaN,...,NaN,NaN,55.0,W,0,0,0,20,40,2
3,002,0.13,0.14,2,420.0,31.0,23665.0,0.09,U,C,...,C,A,55.0,W,0,750,0,27,106,4
4,002,0.14,0.16,2,420.0,31.0,23665.0,0.17,U,C,...,C,A,55.0,W,0,750,0,23,92,4
5,002,0.16,0.17,2,420.0,31.0,23665.0,0.09,U,A,...,C,A,55.0,W,4,350,0,13,52,4
6,002,0.17,0.19,2,420.0,31.0,23665.0,0.17,U,A,...,C,A,55.0,W,4,350,0,13,52,4
7,002,0.19,0.21,2,420.0,31.0,20367.0,0.15,U,A,...,A,A,55.0,W,4,350,10,12,48,4


#### vehicle

In [19]:
veh_file = './hsis-csv/wa17veh.csv'
veh = pd.read_csv(veh_file)

In [20]:
veh.head()

,CASENO,DRV_SEX,DRV_AGE,drassess,spdlimit,surf_typ,contrib1,contrib2,intox,stolen,vehcond1,vehcond2,vehcond3,com_body,cdplaccd,vehno
0,2017000001,1.0,51.0,NaN,60.0,2.0,23,0,4.0,NaN,12,0,0,9.0,F,1
1,2017000001,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,0,NaN,NaN,2
2,2017000002,2.0,30.0,0.0,70.0,1.0,4,0,4.0,NaN,12,0,0,NaN,NaN,1
3,2017000002,1.0,35.0,0.0,60.0,1.0,18,0,4.0,NaN,12,0,0,2.0,F,2
4,2017000002,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,0,NaN,NaN,3


### merge

We need to think of functions, and therefore what joining operations should be present:
+ **crash meta:** accident &#8596; occupant &#8596; vehicle<br>
+ **road meta:** road &#8596; curv &#8596; grade<br>
+ **crash meta** &#8596; **road meta** for analysis

#### YYYY - execute - crash meta
Since one crash may have multiple vehicles and occupants, and that data cannot be easily aggregated (e.g. averaged).<br>
We only count the number of vehicles and add that as an attribute.<br>
+ For example, in **CASENO: 2017000001**, one vehicle has driver info while the other does not.<br>
+ In other cases, both or all vehicles have driver info.<br>

You should have 55494 records in **acc_meta**.

In [21]:
veh_count = veh['CASENO'].value_counts().sort_index()

In [22]:
veh_count = veh_count.to_frame().reset_index()
veh_count.columns = ['CASENO', 'veh_count']

In [23]:
veh_count.shape

(55548, 2)

In [24]:
acc_meta = acc.merge(veh_count, on='CASENO', how='inner')
print(acc.shape, acc_meta.shape)
# .merge(occ, on='CASENO', how='inner')

(55494, 16) (55494, 17)


In [25]:
len(acc_meta['CASENO'].unique())

55494

#### XXXX - don't execute - road meta
ROAD mileposts are in smaller segments, while records in CURVE and GRADE may cover larger area.

It seems we cannot do pre-merge:<br>
+ GRADE and CURVE's segments are cross-overlapping with ROAD's;
+ E.g. **ROAD: [0.16, 0.19]**, **GRADE: [0.16, 0.18]**, **CURV: [0.16, 0.17]**

#### YYYY - execute here - super merge
acc_meta &#8596; road &#8596; curve &#8596; grade

##### XXXX - don't execute - initial merge in sql with ">" and "<" for **crash** and **road**
**found that many records are lost with "<" and ">"**, e.g. *CASENO "2017000003"*

In [ ]:
conn = sqlite3.connect(":memory:")
acc_meta.to_sql("crash", conn, index=False)
road.to_sql("road", conn, index=False)

In [ ]:
query = \
    "SELECT * \
    FROM crash, road \
    WHERE crash.rd_inv = road.ROAD_INV\
    AND crash.milepost > road.BEGMP\
    AND crash.milepost < road.ENDMP"
tt = pd.read_sql_query(query,conn)

In [ ]:
tt.head()

In [ ]:
tt[tt['CASENO'] == 2017000003]

###### But actually the road with the "ROAD_INV" exists

In [ ]:
road[road['ROAD_INV'] == '195']

###### It's because "2017000003"'s milepost is actually at a separating point
There are two records of "2017000003".

In [ ]:
query = \
    "SELECT * \
    FROM crash, road \
    WHERE crash.rd_inv = road.ROAD_INV\
    AND crash.milepost >= road.BEGMP\
    AND crash.milepost <= road.ENDMP"
tt = pd.read_sql_query(query,conn)
tt.head(8)

###### we can also inspect the schema of the two tables

In [ ]:
cur = conn.cursor()
cur.execute('PRAGMA TABLE_INFO({})'.format('crash'))
names = [tup for tup in cur.fetchall()]
print(names)

In [ ]:
cur.execute('PRAGMA TABLE_INFO({})'.format('road'))
names = [tup for tup in cur.fetchall()]
print(names)

In [ ]:
road[(road['ROAD_INV'] == '195') & (road['BEGMP'] < 36) & (road['ENDMP'] > 36)]

##### XXXX - don't execute - do again in sql with ">=" and  "<" for **crash** and **road**

##### XXXX - don't execute - do again in sql with ">=" and  "<=" for **crash** and **road**
+ Remember **acc_meta** has 55494 records;
+ Dataframe **records** has 61765 entries, seems that 61765 - 55494 = 6271 records are duplicated. The reason is that some accidents happen at the seperation points of road segments, i.e. at the milepost that belong to two segments.
+ However, checking unique CASENO of accidents, I found that a lot of accidents are filtered if we do a INNER JOIN;
+ This means some accidents do not have an associated road segment????

In [ ]:
conn = sqlite3.connect(":memory:")
acc_meta.to_sql("crash", conn, index=False)
road.to_sql("road", conn, index=False)

In [ ]:
len(acc_meta['CASENO'].unique())

In [ ]:
query = \
    "SELECT * \
    FROM crash LEFT JOIN road ON crash.rd_inv = road.ROAD_INV\
    AND crash.milepost >= road.BEGMP\
    AND crash.milepost <= road.ENDMP"

records = pd.read_sql_query(query, conn)

###### Using LEFT JOIN everything is better
+ Because the CASENO has 55494 unique values, same with **acc_meta**.
+ But a lot of crashes do not have an associated road

In [ ]:
len(records['CASENO'].unique())

+ As noted, there is a 1729 difference in the key "ROAD_INV" in **acc_meta** and **road**. Why?<br>
+ Some accident roadways are not in the road file???

In [ ]:
len(set(list(records['rd_inv'].unique())) - set(list(records['ROAD_INV'].unique())))

Let's inspect three
+ Randomly choose three which are recorded in crashes but not in road inventories;
+ Retrieve the crash and road record, respectively;

In [ ]:
random.sample(set(list(records['rd_inv'].unique())) - set(list(records['ROAD_INV'].unique())), 3)

In [ ]:
records[records['rd_inv'] == '005LX10523']

###### check what accidents don't have road info

In [ ]:
acc_file = './hsis/wa17acc.xlsx'
acc = pd.read_excel(acc_file)

In [ ]:
acc[acc['rd_inv'] == '005LX10523']

##### YYYY - execute - We still apply the inner join for **crash** and **road**
INNER JOIN and then remove duplicates

In [26]:
road = road.drop(['RTE_NBR'], axis=1)

conn = sqlite3.connect(":memory:")
acc_meta.to_sql("crash", conn, index=False)
road.to_sql("road", conn, index=False)

In [27]:
query = \
    "SELECT * \
    FROM crash, road\
    WHERE crash.rd_inv = road.ROAD_INV\
    AND crash.milepost >= road.BEGMP\
    AND crash.milepost <= road.ENDMP"

records = pd.read_sql_query(query, conn)

In [28]:
records.shape

(61983, 37)

##### YYYY - execute - remove the duplicates randomly
The final result has 47818 records

In [29]:
records = records.sample(frac=1).drop_duplicates(subset='CASENO').sort_index()

In [30]:
records.columns

Index(['CASENO', 'FORM_REPT_NO', 'rd_inv', 'milepost', 'RTE_NBR', 'lat', 'lon',
       'MONTH', 'DAYMTH', 'WEEKDAY', 'RDSURF', 'LIGHT', 'weather', 'rur_urb',
       'REPORT', 'SEVERITY', 'veh_count', 'ROAD_INV', 'BEGMP', 'ENDMP', 'CITY',
       'COUNTY', 'AADT', 'mvmt', 'RURURB', 'LSHL_TYP', 'MED_TYPE', 'RSHL_TYP',
       'SURF_TYP', 'SPD_LIMT', 'EW_IND', 'LSHLDWID', 'MEDWID', 'RSHLDWID',
       'lanewid', 'rdwy_wid', 'NO_LANES'],
      dtype='object')

#### YYYY - execute - connect with curve
Here we can use left join because no match means zero curvature

In [31]:
conn = sqlite3.connect(":memory:")
records.to_sql("records", conn, index=False)
curv.to_sql("curv", conn, index=False)

In [32]:
query = \
    "SELECT * \
    FROM records LEFT JOIN curv ON records.rd_inv = curv.curv_inv\
    AND records.milepost >= curv.begmp\
    AND records.milepost <= curv.endmp"

In [33]:
records = pd.read_sql_query(query, conn)

In [34]:
records = records.sample(frac=1).drop_duplicates(subset='CASENO').sort_index()
records = records.drop(['curv_inv', 'begmp', 'endmp', 'rte_nbr', 'DIR_CURV'], axis=1)

In [35]:
records = records.fillna(value={'deg_curv': 0})

#### YYYY - execute - connect with grade
Can we use LEFT JOIN here?
+ LEFT JOIN: can be Nan, which means 0 grade;
+ INNER JOIN: must have something

In [36]:
conn = sqlite3.connect(":memory:")
records.to_sql("records", conn, index=False)
grad.to_sql("grad", conn, index=False)

In [37]:
query = \
    "SELECT * \
    FROM records LEFT JOIN grad ON records.rd_inv = grad.grad_inv\
    AND records.milepost >= grad.begmp\
    AND records.milepost <= grad.endmp"

In [38]:
records = pd.read_sql_query(query, conn)

In [39]:
records = records.drop(['grad_inv', 'begmp', 'endmp', 'rte_nbr'], axis=1)
records = records.sample(frac=1).drop_duplicates(subset='CASENO').sort_index()

In [40]:
records = records.fillna(value={'pct_grad': 0})

In [41]:
records

,CASENO,FORM_REPT_NO,rd_inv,milepost,RTE_NBR,lat,lon,MONTH,DAYMTH,WEEKDAY,...,EW_IND,LSHLDWID,MEDWID,RSHLDWID,lanewid,rdwy_wid,NO_LANES,deg_curv,dir_grad,pct_grad
0,2017000001,E627527,005,64.13,5,46.484755,-122.879959,1,1,7,...,W,3,40,10,12,48,4,0.00,+,3.00
1,2017000002,E628124,090,70.58,90,47.228310,-121.163097,1,2,1,...,E,4,78,10,12,48,4,0.00,-,0.15
2,2017000003,E630351,195,35.95,195,46.879918,-117.364820,1,9,1,...,E,0,0,0,14,56,4,0.00,-,0.49
3,2017000004,E633194,270,3.13,270,46.726902,-117.168536,1,9,1,...,E,0,0,0,18,72,4,0.00,+,3.86
4,2017000005,E632431,014,139.99,14,45.787757,-120.099029,1,14,6,...,E,7,0,7,12,24,2,0.00,+,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50889,2017055645,E774350,005,127.45,5,47.436457,-122.335086,12,15,5,...,W,0,16,10,12,96,8,0.00,+,1.69
50890,2017055647,E774336,005,134.98,5,47.221556,-122.463305,12,16,6,...,W,0,16,10,12,84,7,0.82,-,1.78
50891,2017055648,E774338,005,130.74,5,47.376333,-122.299836,12,16,6,...,W,0,16,10,12,96,8,0.00,-,0.20
50892,2017055650,E774290,005,133.86,5,47.467451,-122.223553,12,18,1,...,W,0,16,0,16,109,7,0.00,-,3.00


In [42]:
records.to_csv('meta_merged.csv')

### auto - conversion
but too slow

In [47]:
acc = acc.dropna(subset=['X_full', 'Y_full']).reset_index()

In [48]:
acc['lat'] = None
acc['lng'] = None

In [55]:
start = time.time()
for index, row in acc.iterrows():
    north = row['Y_full']
    east = row['X_full']
    
    j = !java -Dparms=spc,4602,$north,$east,usft,"NAD83(2011)","NAD83(2011)","auto","N/A" -jar ./ncat/jtransform_thin.jar
    lat, lng = j[7][:-1].split(':')[-1][1:-1], j[12][:-1].split(':')[-1][1:-1]
    row['lat'] = lat
    row['lng'] = lng
    
    if index == 100:
        break
print("{:.2f}".format(time.time() - start))

22.88
